In [46]:
from models.edenTTS import EdenTTS
from hparams import hparams as hp
from utils.paths import Paths
import torch
import time
import soundfile as sf
import os
from utils.log_util import get_logger
import numpy as np
from text.en_util import text_to_sequence
from qwen_tts import Qwen3TTSTokenizer
import soundfile as sf
# THÊM THƯ VIỆN ĐỂ NGHE AUDIO TRỰC TIẾP TRÊN JUPYTER
from IPython.display import Audio, display 

device = "cuda" if torch.cuda.is_available() else "cpu"
log = get_logger(__name__)

In [2]:
# ==========================================
# 1. LOAD TOKENIZER VÀ MODEL (Chỉ cần chạy 1 lần)
# ==========================================
log.info("Loading Qwen3 TTS Tokenizer...")
tokenizer = Qwen3TTSTokenizer.from_pretrained(
    "Qwen/Qwen3-TTS-Tokenizer-12Hz",
    device_map=device,
)

2026-02-20 09:07:32,108 INFO 2871649631.py-4 Loading Qwen3 TTS Tokenizer...


In [ ]:
# Khởi tạo và load trọng số EdenTTS
tts_model_id = hp.tts_model_id
paths = Paths(hp.data_path, tts_model_id, speaker=hp.speaker)
tts_model_path = paths.tts_latest_weights
print(f"Đường dẫn checkpoint: {tts_model_path}")

if not os.path.exists(tts_model_path):
    print(f"⚠️ LỖI: Không tìm thấy file checkpoint tại: {tts_model_path}")
else:
    tts_model = EdenTTS(hp).to(device)
    tts_model.load(tts_model_path)
    tts_model.eval() # Bắt buộc set eval mode
    log.info("EdenTTS Model loaded successfully!")

out_path = "./"
os.makedirs(out_path, exist_ok=True)

In [52]:
enc = tokenizer.encode("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-TTS-Repo/tokenizer_demo_1.wav")
OutputClass = enc.__class__

In [53]:
# ==========================================
# 2. ĐỊNH NGHĨA HÀM SUY LUẬN
# ==========================================
def synthesize_text(text):
    if not os.path.exists(tts_model_path):
        return
        
    log.info(f"Processing: '{text}'")
    phones = text_to_sequence(text, token_type=hp.token_type) 
    phones = torch.tensor(phones).long().unsqueeze(0).to(device)
    
    s1 = time.time()
    # Dự đoán Latent vector liên tục
    mel_pred = tts_model.inference(phones)
    log.info(f"Acoustic model inference time: {time.time() - s1:.3f}s")
    safe_name = "".join([c if c.isalnum() else "_" for c in text[:15]])
    file_name = f"{safe_name}_{int(time.time())}.wav"
    file_path = os.path.join(out_path, file_name)
    with torch.no_grad():
        # De-normalize: Nhân 2048, làm tròn, ép kiểu integer và kẹp trong khoảng codebook
        tokens = torch.round(mel_pred * 2048).long()
        tokens = torch.clamp(tokens, min=0, max=2047)
        new_output = OutputClass(audio_codes=[tokens[0]])
        print(new_output)
        # print({"audio_codes": [tokens[0]]})
        
        # Bọc tensor vào dictionary với key "audio_codes"
        wav, sr = tokenizer.decode(new_output)
        print(wav)
        sf.write("test.wav", wav[0], sr)
    
    log.info(f"Saved at: {file_path}")
    
    # Render thanh Audio ngay trên Jupyter Cell
    display(Audio(wav, rate=sr))

In [ ]:
# Ô CHẠY THỬ NGHIỆM (Chạy bao nhiêu lần tùy thích)
text_to_test = "in being comparatively modern."
synthesize_text(text_to_test)